<a href="https://colab.research.google.com/github/amanikonda123/CookbookWizard.ai/blob/main/part1_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import collections
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, Input
from tensorflow.keras import backend as K

In [2]:
!pip install --upgrade kaggle

In [3]:
!pip install opendatasets

In [4]:
import opendatasets as od
od.download(
    "https://www.kaggle.com/competitions/ifood-2019-fgvc6/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: adityamanikonda
Your Kaggle Key: ··········


100%|██████████| 2.84G/2.84G [00:33<00:00, 91.4MB/s]



Extracting archive ./ifood-2019-fgvc6/ifood-2019-fgvc6.zip to ./ifood-2019-fgvc6


In [5]:
#unzipping the zip files and deleting the zip files
!unzip /content/ifood-2019-fgvc6/train_set.zip
!unzip /content/ifood-2019-fgvc6/test_set.zip
!unzip /content/ifood-2019-fgvc6/val_set.zip

Streaming output truncated to the last 5000 lines.
  inflating: val_set/val_006506.jpg  
  inflating: val_set/val_004199.jpg  
  inflating: val_set/val_000848.jpg  
  inflating: val_set/val_007717.jpg  
  inflating: val_set/val_007130.jpg  
  inflating: val_set/val_002708.jpg  
  inflating: val_set/val_001085.jpg  
  inflating: val_set/val_005015.jpg  
  inflating: val_set/val_012038.jpg  
  inflating: val_set/val_010613.jpg  
  inflating: val_set/val_010968.jpg  
  inflating: val_set/val_007924.jpg  
  inflating: val_set/val_010303.jpg  
  inflating: val_set/val_001124.jpg  
  inflating: val_set/val_011892.jpg  
  inflating: val_set/val_006131.jpg  
  inflating: val_set/val_012008.jpg  
  inflating: val_set/val_004498.jpg  
  inflating: val_set/val_008342.jpg  
  inflating: val_set/val_005417.jpg  
  inflating: val_set/val_012107.jpg  
  inflating: val_set/val_002339.jpg  
  inflating: val_set/val_010129.jpg  
  inflating: val_set/val_000308.jpg  
  inflating: val_set/val_004271.jpg  

In [6]:
df_train_labels = pd.read_csv("/content/ifood-2019-fgvc6/train_labels.csv")
df_train_labels['label'] = df_train_labels['label'].astype(str)

df_val_labels = pd.read_csv("/content/ifood-2019-fgvc6/val_labels.csv")
df_val_labels['label'] = df_val_labels['label'].astype(str)

num_classes = df_train_labels['label'].nunique()

In [7]:
df_class_list = pd.read_csv('/content/ifood-2019-fgvc6/class_list.txt')
df_class_list.rename(columns={'0 macaron': 'Image Class'}, inplace=True)
df_class_list['Image Class'] = df_class_list['Image Class'].str.replace(r'^\d+\s+', '', regex=True)

In [8]:
# class DropBlock(Layer):
#     def __init__(self, block_size=3, keep_prob=0.9, **kwargs):
#         super(DropBlock, self).__init__(**kwargs)
#         self.block_size = block_size
#         self.keep_prob = keep_prob

#     def call(self, inputs, training=None):
#         if training:
#             gamma = (1.0 - self.keep_prob) * (self.feat_size ** 2) / (self.block_size ** 2) / ((self.feat_size - self.block_size + 1) ** 2)
#             mask = tf.random.uniform(shape=[self.feat_size // self.block_size, self.feat_size // self.block_size], minval=0, maxval=1) < gamma
#             mask = tf.image.resize(mask[..., tf.newaxis], [self.feat_size, self.feat_size], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#             mask = tf.squeeze(mask, axis=-1)
#             mask = 1 - mask  # Invert the mask: 1 for keep, 0 for drop
#             mask = tf.tile(mask[..., tf.newaxis], [inputs.shape[0], 1, 1, inputs.shape[3]])  # Repeat the mask for each batch and channel
#             return inputs * mask * (1.0 / self.keep_prob)
#         else:
#             return inputs


In [9]:
class MixupImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator, dataframe, directory, batch_size, img_height, img_width, alpha=0.2, num_classes=251):
        self.batch_size = batch_size
        self.generator = generator.flow_from_dataframe(
            dataframe=dataframe,
            directory=directory,
            x_col="img_name",
            y_col="label",
            target_size=(img_height, img_width),
            batch_size=batch_size,
            class_mode="categorical")
        self.alpha = alpha
        self.num_classes = num_classes

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, idx):
        x, y = next(self.generator)
        batch_size = x.shape[0]

        # Perform Mixup
        lam = np.random.beta(self.alpha, self.alpha, size=batch_size)
        index_array = np.random.permutation(batch_size)

        mixed_x = lam.reshape(batch_size, 1, 1, 1) * x + (1 - lam).reshape(batch_size, 1, 1, 1) * x[index_array]
        mixed_y = lam.reshape(batch_size, 1) * y + (1 - lam).reshape(batch_size, 1) * y[index_array]

        return mixed_x, mixed_y

    def on_epoch_end(self):
        self.generator.on_epoch_end()

In [10]:
subdir = '/content/test_set/images'
os.makedirs(subdir, exist_ok=True)

# Move all images from unzip_dir to subdir
for file_name in os.listdir('/content/test_set'):
    if file_name.endswith(('.png', '.jpg', '.jpeg')):  # Add other file types if needed
        os.rename(os.path.join('/content/test_set', file_name), os.path.join(subdir, file_name))

print("Images moved to subdirectory.")

Images moved to subdirectory.


In [11]:
file_paths = []
file_names = os.listdir('/content/test_set/images')
for file_name in file_names:
    if file_name.endswith(('.png', '.jpg', '.jpeg')):  # Add other file types if needed
        file_path = os.path.join('/content/test_set/images', file_name)
        file_paths.append(file_path)

# Create a DataFrame
df_test = pd.DataFrame(file_paths, columns=['filename'])
print(df_test.head())

                                   filename
0  /content/test_set/images/test_020557.jpg
1  /content/test_set/images/test_017896.jpg
2  /content/test_set/images/test_022796.jpg
3  /content/test_set/images/test_016973.jpg
4  /content/test_set/images/test_019414.jpg


In [12]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_mixup_generator = MixupImageDataGenerator(
    generator=train_datagen,
    dataframe=df_train_labels,
    directory='/content/train_set',
    batch_size=64,
    img_height=224,
    img_width=224,
    alpha=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=df_val_labels,
    directory='/content/val_set',
    x_col='img_name',
    y_col='label',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='filename',
    y_col=None,  # No label column
    target_size=(224, 224),
    color_mode='rgb',
    class_mode=None,  # No labels
    batch_size=64,
    shuffle=False
)

Found 118475 validated image filenames belonging to 251 classes.
Found 11994 validated image filenames belonging to 251 classes.
Found 28377 validated image filenames.


In [13]:
# Callbacks
callbacks = [
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)
]

In [14]:
# Base model - ResNet50
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_resnet = base_model_resnet.output
x_resnet = GlobalAveragePooling2D()(x_resnet)
x_resnet = Dense(1024, activation='relu')(x_resnet)
predictions_resnet = Dense(num_classes, activation='softmax')(x_resnet)
model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions_resnet)

# Compile models
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 1s 0us/step


In [15]:
# Training - ResNet50
history_resnet = model_resnet.fit(
    train_mixup_generator,
    epochs=1,
    validation_data=val_generator,
    callbacks=callbacks
)

1852/1852 [==============================] - ETA: 0s - loss: 3.8226 - accuracy: 0.2443

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1852/1852 [==============================] - 2456s 1s/step - loss: 3.8226 - accuracy: 0.2443 - val_loss: 2.6639 - val_accuracy: 0.3799 - lr: 1.0000e-04


In [16]:
# Base model - InceptionV3
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_inception = base_model_inception.output
x_inception = GlobalAveragePooling2D()(x_inception)
x_inception = Dense(1024, activation='relu')(x_inception)
predictions_inception = Dense(num_classes, activation='softmax')(x_inception)
model_inception = Model(inputs=base_model_inception.input, outputs=predictions_inception)

model_inception.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


87910968/87910968 [==============================] - 0s 0us/step


In [17]:
# Training - InceptionV3
history_inception = model_inception.fit(
    train_mixup_generator,
    epochs=1,
    validation_data=val_generator,
    callbacks=callbacks
)

1852/1852 [==============================] - 2228s 1s/step - loss: 3.9167 - accuracy: 0.2297 - val_loss: 2.4689 - val_accuracy: 0.4159 - lr: 1.0000e-04


In [18]:
def ensemble_predictions(models, generator, weights=None):
    """Generate ensemble predictions from multiple models."""
    if weights is None:
        # Assign equal weight to all models if no specific weights are provided
        weights = [1. / len(models)] * len(models)

    total_predictions = None
    for model, weight in zip(models, weights):
        predictions = model.predict(generator)
        if total_predictions is None:
            total_predictions = predictions * weight
        else:
            total_predictions += predictions * weight

    return total_predictions

# List of models
models = [model_resnet, model_inception]

# Generate ensemble predictions
# Assuming 'test_generator' is your test data generator
ensemble_pred = ensemble_predictions(models, test_generator)

# Convert probabilities to class indices
final_predictions = np.argmax(ensemble_pred, axis=1)


444/444 [==============================] - 91s 202ms/step


In [19]:
final_predictions

array([204, 127, 231, ..., 229, 198, 245])

In [20]:
df_class_list = pd.read_csv('/content/ifood-2019-fgvc6/class_list.txt')
df_class_list.rename(columns={'0 macaron': 'Image Class'}, inplace=True)
df_class_list['Image Class'] = df_class_list['Image Class'].str.replace(r'^\d+\s+', '', regex=True)
df_class_list

,Image Class
0,beignet
1,cruller
2,cockle_food
3,samosa
4,tiramisu
...,...
245,chicken_cordon_bleu
246,eccles_cake
247,moo_goo_gai_pan
248,buffalo_wing


In [21]:
predicted_class_names = df_class_list['Image Class'].iloc[final_predictions]
print(min(final_predictions))

IndexError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# Generate ensemble predictions

predicted_classes = np.argmax(ensemble_pred, axis=1)

# You need to have a list or array of true labels for your test set
# true_labels = [...]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_classes)
print(f"Ensemble Test Accuracy: {accuracy*100:.2f}%")

# Generate confusion matrix
cm = confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:")
print(cm)

# Generate a classification report
report = classification_report(true_labels, predicted_classes)
print("\nClassification Report:")
print(report)
